# Extract the data from of the balances finacieros pdfs in Mongodb
### Connect to mongodb and make sure we have access to the pdf files

In [5]:
packages = ['pymongo', 'gridfs', 'concurrent.futures']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
from gridfs import GridFS
from concurrent.futures import ThreadPoolExecutor


endpoint = '10.0.10.5:27017'
database = 'supercias'
collection = 'companies'

# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
collection = db[collection]

#find one
comp = collection.find_one()

docs = comp['Documentos online']['downloaded']

pdfs = []
#if company has documents
for value in docs.values():
    if isinstance(value, list) and len(value) > 0:
        [ pdfs.append(v) for v in value ]

print(pdfs)

['0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1994-09-13.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1996-09-06.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2008-05-08.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2009-01-14.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2013-05-13.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2013-07-17.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2013-11-12.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2020-10-23.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2022-03-22.pdf', '0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2023-11-21.pdf', '0991295437001_DocumentosGenerales_Oficio Nombramiento Administradores_1994-09-09_CORDOVEZ ORTEGA JUAN XAVIER_PRESIDENTE.pdf', '0991295437001_DocumentosGenerales_Oficio Nombramien

### Read from errors files

In [6]:
 # read from error.log file and load the content into a list

with open('error.log', 'r') as file:
    error_list = file.readlines()

# trim the newline character from the end of each line
error_list = [line.strip() for line in error_list]

print(error_list)

['0190049531001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190167275001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190335100001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190049531001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190167275001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190335100001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190376982001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190390241001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190406806001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190429989001_DocumentosEconomicos_Balance  Estado de Situación Financiera_2023-12-31.pdf', '0190437604001_DocumentosEconomicos_Balance  Estado de Situación Fina

#### Query the files from the mondog db

In [7]:
# Access the GridFS collection
fs = GridFS(db, collection='companies')

error_list = ['1790016919001_DocumentosGenerales_Oficio Transferencia Acciones_1988-10-17.pdf']

def query_files(filename):
    # Retrieve all files from GridFS collection that have in the filen ethe substring 'Balance  Estado de Situación'
    pdf_queried = fs.find({'filename': {'$regex': filename}})
    return pdf_queried

# for every error_list query the file
queries = [ query_files(filename) for filename in error_list ]

print(queries)

#### Query the pdfs and write them 

In [8]:
# track the time taken to read a file from GridFS and write it to the disk
path = '../storage/pdfs/'

# make dir path if it does not exist
if not os.path.exists(path):
    os.makedirs(path)

def write_pdf(pdf_query):
    pdf_query = pdf_query[0]
    buffer = pdf_query.read()
    print('buffer length:', len(buffer))
    # write the file to the disk
    print('writing:', pdf_query.filename)
    with open( path + pdf_query.filename, 'wb') as f:
        f.write(buffer)
    print('done writing:', pdf_query.filename)

# for every query, get the first file and write it to the disk
for query in queries:
    write_pdf(query)

buffer length: 1165329
writing: 1790016919001_DocumentosGenerales_Oficio Transferencia Acciones_1988-10-17.pdf
done writing: 1790016919001_DocumentosGenerales_Oficio Transferencia Acciones_1988-10-17.pdf
